In [ ]:
import numpy as np
import math
from scipy.special import erfc

def universal_test(n, epsilon):
    expected_value = [0, 0, 0, 0, 0, 0, 5.2177052, 6.1962507, 7.1836656,
                      8.1764248, 9.1723243, 10.170032, 11.168765,
                      12.168070, 13.167693, 14.167488, 15.167379]

    variance = [0, 0, 0, 0, 0, 0, 2.954, 3.125, 3.238, 3.311, 3.356, 3.384,
                3.401, 3.410, 3.416, 3.419, 3.421]

    # Determine L value
    L = 2
    if n >= 387840: L = 6
    if n >= 904960: L = 7
    if n >= 2068480: L = 8
    if n >= 4654080: L = 9
    if n >= 10342400: L = 10
    if n >= 22753280: L = 11
    if n >= 49643520: L = 12
    if n >= 107560960: L = 13
    if n >= 231669760: L = 14
    if n >= 496435200: L = 15
    if n >= 1059061760: L = 16

    Q = 10 * (2 ** L)
    K = math.floor(n / L) - Q  # Blocks to test

    p = 2 ** L
    if (L < 6 or L > 16 or Q < 10 * (2 ** L)):
        print("ERROR: L is out of range or Q is too small.")
        return

    # Compute expected value
    c = 0.7 - 0.8 / L + (4 + 32 / L) * (K ** (-3 / L)) / 15
    sigma = c * math.sqrt(variance[L] / K)
    sqrt2 = math.sqrt(2)
    sum_values = 0.0

    T = np.zeros(p, dtype=int)

    # Initialize Table
    for i in range(1, Q + 1):
        decRep = sum(epsilon[(i - 1) * L + j] * (2 ** (L - 1 - j)) for j in range(L))
        T[decRep] = i

    # Process Blocks
    for i in range(Q + 1, Q + K + 1):
        decRep = sum(epsilon[(i - 1) * L + j] * (2 ** (L - 1 - j)) for j in range(L))
        sum_values += math.log(i - T[decRep], 2)
        T[decRep] = i

    phi = sum_values / K

    print("\nUNIVERSAL STATISTICAL TEST")
    print("---------------------------------------------")
    print(f"(a) L         = {L}")
    print(f"(b) Q         = {Q}")
    print(f"(c) K         = {K}")
    print(f"(d) sum       = {sum_values}")
    print(f"(e) sigma     = {sigma}")
    print(f"(f) variance  = {variance[L]}")
    print(f"(g) exp_value = {expected_value[L]}")
    print(f"(h) phi       = {phi}")
    print(f"(i) WARNING: {n - (Q + K) * L} bits were discarded.")
    print("---------------------------------------------")

    arg = abs(phi - expected_value[L]) / (sqrt2 * sigma)
    p_value = erfc(arg)

    if p_value < 0 or p_value > 1:
        print("WARNING: P_VALUE IS OUT OF RANGE")

    print(f"{'FAILURE' if p_value < 0.01 else 'SUCCESS'}\tp_value = {p_value}\n")

    return p_value